<img src="https://unskript.com/assets/favicon.png" alt="unSkript.com" width="100" height="100"/> 
<h1> unSkript Runbooks </h1>
<div class="alert alert-block alert-success">
    <b> This runbook demonstrates How to Detach EC2 Instance from ASG and Load Balancer using unSkript legos.</b>
</div>

<br>

<center><h2>Detach EC2 Instance from ASG and Load Balancer</h2></center>

# Steps Overview
1) List all the instances available in the given region.
2) Get the instances which are attached to Auto Scalling Group.
3) Detach the given instance if available in the Auto Scalling Group.
4) Deregisters AWS EC2 Instances from a given Load Balancer.

Here we will use unSkript Filter All AWS EC2 Instances Lego. This lego take region: str as input. This inputs is used to findout all the AWS EC2 instances.

In [11]:
##
##  Copyright (c) 2021 unSkript, Inc
##  All rights reserved.
##
from pydantic import BaseModel, Field
from typing import List
from unskript.connectors.aws import aws_get_paginator
import pprint

from beartype import beartype
@beartype
def aws_filter_ec2_instances_printer(output):
    if output is None:
        return
    pprint.pprint({"Instances": output})


@beartype
def aws_filter_ec2_instances(handle, region: str) -> List:
    """aws_filter_ec2_instances Returns an array of instances.

        :type handle: object
        :param handle: Object returned from task.validate(...).

        :type region: string
        :param region: Region to filter instances.

        :rtype: Array of instances.
    """
    ec2Client = handle.client('ec2', region_name=region)
    res = aws_get_paginator(ec2Client, "describe_instances", "Reservations")
    result = []
    for reservation in res:
        for instance in reservation['Instances']:
            result.append(instance['InstanceId'])
    return result


task = Task(Workflow())
task.configure(credentialsJson='''{
    "credential_name": "DevRole",
    "credential_type": "CONNECTOR_TYPE_AWS",
    "credential_id": "0b438eba-0627-4f6d-b998-a4c604f20e3c"
}''')
task.configure(inputParamsJson='''{
    "region": "Region"
    }''')
task.configure(outputName="InstanceIDs")

(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_filter_ec2_instances, lego_printer=aws_filter_ec2_instances_printer, hdl=hdl, args=args)

Here we will use unSkript Get AWS Auto Scalling Group Instances Lego. This lego take instance_ids: list and region: str as inputs. This inputs is used to findout all the AWS EC2 instances attached to Auto Scalling Group.

In [12]:
##  Copyright (c) 2021 unSkript, Inc
##  All rights reserved.
##
from typing import List, Dict
from pydantic import BaseModel, Field
from tabulate import tabulate
from botocore.exceptions import ClientError
import pprint

from beartype import beartype
@beartype
def aws_get_auto_scaling_instances_printer(output):
    if output is None:
        return
    print(tabulate(output, headers='keys'))


@beartype
def aws_get_auto_scaling_instances(handle, instance_ids: list, region: str) -> List:
    """aws_get_auto_scaling_instances List of Dict with instanceId and attached groups.

        :type handle: object
        :param handle: Object returned from task.validate(...).

        :type instance_ids: list
        :param instance_ids: List of instances.

        :type region: string
        :param region: Region to filter instances.

        :rtype: List of Dict with instanceId and attached groups.
    """
    result = []
    ec2Client = handle.client('autoscaling', region_name=region)
    try:
        response = ec2Client.describe_auto_scaling_instances(InstanceIds=instance_ids)
        for group in response["AutoScalingInstances"]:
            group_dict = {}
            group_dict["InstanceId"] = group["InstanceId"]
            group_dict["AutoScalingGroupName"] = group["AutoScalingGroupName"]
            result.append(group_dict)
    except Exception as error:
        err = {"Error":error}
        result.append(err)
    return result


task = Task(Workflow())
task.configure(credentialsJson='''{
    "credential_name": "DevRole",
    "credential_type": "CONNECTOR_TYPE_AWS",
    "credential_id": "0b438eba-0627-4f6d-b998-a4c604f20e3c"
}''')
task.configure(inputParamsJson='''{
    "instance_ids": "InstanceIDs",
    "region": "Region"
    }''')
task.configure(outputName="ASGInstances")

(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_get_auto_scaling_instances, lego_printer=aws_get_auto_scaling_instances_printer, hdl=hdl, args=args)

Here we will use unSkript Detach AWS Instance From ASG Lego. This lego take  instance_id: str, group_name: str, region: str as inputs. This inputs is used to remove AWS instance from Auto Scalling Group.

In [22]:
##  Copyright (c) 2021 unSkript, Inc
##  All rights reserved.
##
from typing import List, Dict
from pydantic import BaseModel, Field
import pprint

from beartype import beartype
@beartype
def aws_detach_autoscaling_instances_printer(output):
    if output is None:
        return
    pprint.pprint(output)


@beartype
def aws_detach_autoscaling_instances(
    handle,
    instance_ids: str,
    group_name: str,
    region: str
) -> Dict:
    """aws_detach_autoscaling_instances detach instances from autoscaling group.

        :type handle: object
        :param handle: Object returned from task.validate(...).

        :type instance_ids: string
        :param instance_ids: Name of instances.

        :type group_name: string
        :param group_name: Name of AutoScaling Group.

        :type region: string
        :param region: AWS Region of autoscaling group.

        :rtype: Dict with the detach instance info.
    """

    ec2Client = handle.client("autoscaling", region_name=region)
    result = {}
    try:
        if instance_ids in Instance_IDs:
            response = ec2Client.detach_instances(
                InstanceIds=[instance_ids],
                AutoScalingGroupName=group_name,
                ShouldDecrementDesiredCapacity=True
                )
            result = response
        else:
            result["error"] = "Instance not match"
    except Exception as error:
        result["error"] = error

    return result


task = Task(Workflow())
task.configure(continueOnError=True)
task.configure(credentialsJson='''{
    "credential_name": "DevRole",
    "credential_type": "CONNECTOR_TYPE_AWS",
    "credential_id": "0b438eba-0627-4f6d-b998-a4c604f20e3c"
}''')
task.configure(inputParamsJson='''{
    "group_name": "iter.get(\\"AutoScalingGroupName\\")",
    "instance_ids": "iter.get(\\"InstanceId\\")",
    "region": "Region"
    }''')
task.configure(iterJson='''{
    "iter_enabled": true,
    "iter_list_is_const": false,
    "iter_list": "ASGInstances",
    "iter_parameter": ["group_name","instance_ids"]
    }''')
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_detach_autoscaling_instances, lego_printer=aws_detach_autoscaling_instances_printer, hdl=hdl, args=args)

In this Runbook, we demonstrated the use of unSkript's AWS legos to Detach AWS instance from ASG and ELB. This runbook help to detach the given instance from Auto Scalling Group and given Load Balancer. To view the full platform capabilities of unSkript please visit https://unskript.com